# GNN training

This notebook is used to train a GNN for the inorder traversal of a binary tree.

In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import pickle, os

In [2]:
dataset_name = "balanced-shuffled"
path_dataset = f"./data/{dataset_name}"
path_dataset_pkl = f"{path_dataset}/dataset.pkl"

In [15]:
# load dataset
with open(path_dataset_pkl, "rb") as f : dataset = pickle.load(f)

In [16]:
import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

In [17]:
embedding_size = 32
seed = 42
inp_features = 1
out_features = dataset[0].y.shape[0]

print(f"Input features  : {inp_features}")
print(f"Output features : {out_features}")

Input features  : 1
Output features : 10


In [18]:
class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(seed)

        # GCN layers
        self.initial_conv = GCNConv(inp_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size*2, out_features)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(1, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (out): Linear(in_features=64, out_features=10, bias=True)
)
Number of parameters:  3882


### Train

In [9]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(torch.cuda.is_available(), device)

False cpu


In [19]:
dataset_size = len(dataset)
batch_size = 64
trn_loader = DataLoader(dataset[:int(dataset_size * 0.8)], batch_size=batch_size, shuffle=True)
tst_loader = DataLoader(dataset[int(dataset_size * 0.8):], batch_size=batch_size, shuffle=True)

In [24]:
def train(loader, device):
    for batch in loader:
        batch.to(device)
        optimizer.zero_grad()
        print(batch.x.shape, batch.edge_index.shape, batch.batch.shape)
        pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch)
        loss = loss_fn(pred, batch.y)
        loss.backward()
        optimizer.step()
    return loss, embedding

In [25]:
print("Starting training...")
losses = []
for epoch in range(2000):
    loss, h = train(trn_loader, device)
    losses.append(loss)
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Train Loss {loss}")

Starting training...
torch.Size([640, 1]) torch.Size([9, 128]) torch.Size([640])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 9 but got size 2 for tensor number 1 in the list.